In [46]:
import numpy as np
import scipy.signal
from scipy.optimize import minimize
import pandas as pd
pd.set_option('max_columns', None)
from sklearn import linear_model
import matplotlib.pyplot as plt
from datetime import datetime
plt.rcParams['figure.figsize'] = [15, 7.5]
plt.rcParams.update({'font.size': 16,'legend.fontsize':16})
import seaborn as sns
import sys
import os
import fur.path_assistant as path_assistant
from fur.waveform_reader import read_waveform
from fur.finding_period import get_period
from fur.fluctuations import get_fluctiation_and_noise_var
from fur.one_data_set_analyzer import analyze_one_dataset
from fur.extending_with_acnet_data import get_fluctuations_df_with_acnet_data, get_acnet_df_with_bunch_sizes
shift = path_assistant.PathAssistant('shift_03_16_2020',
    ignore_files=['RefCurve_2020-03-16_0_084020_test.Wfm.bin',
                  'RefCurve_2020-03-16_153_155200.Wfm.bin'])
#shift_03_09_2020.show_waveform_file_names()
waveforms_dir = shift.get_waveforms_dir()
results_dir = shift.get_results_dir()
from config import get_from_config, save_to_config
from wiggler_radiation.number_of_coherent_modes.coherent_modes import get_M_interpolator_at_fixed_energy
import lattice.lattice as lattice
from lattice.summary_in_undulator import CalcTransverseBeamParams
from wiggler_radiation.Wigrad.wigrad_generator import \
    generate_wr_sim_with_wigrad_results
import coherent_modes_cpp as cm

In [47]:
cur_to_sum_channel = get_from_config("Beam_current_to_Sum_channel_ampl_V/mA")
sum_channel_to_photoelectrons =\
    get_from_config('sum_channel_to_photoelectrons')
meas_photons_per_electron =\
    get_from_config("Measured_photons_per_electron")
meas_photons_per_electron

0.008820875499988542

In [48]:
from scipy.interpolate import interp1d
bp_df = pd.read_csv("beam_params_vs_current.csv")
sx_func = interp1d(bp_df["N"], bp_df['Sigma_um_X'],
                   bounds_error=False, fill_value="extrapolate")
sy_func = interp1d(bp_df["N"], bp_df['Sigma_um_Y'],
                   bounds_error=False, fill_value="extrapolate")
sz_func = interp1d(bp_df["N"], bp_df['sz_um'],
                   bounds_error=False, fill_value="extrapolate")
ex_func = interp1d(bp_df["N"], bp_df['ex_um'],
                   bounds_error=False, fill_value="extrapolate")
ey_func = interp1d(bp_df["N"], bp_df['ey_um'],
                   bounds_error=False, fill_value="extrapolate")
dpp_func = interp1d(bp_df["N"], bp_df['dp/p'],
                    bounds_error=False, fill_value="extrapolate")

In [49]:
m0 = 10000
mfold = 10
seed = 1

In [50]:
lattice_df = \
    lattice.read_lattice_file(shift.get_6dsim_dir()\
    .fi("IOTA_1NL_100MeV_v8.6.1.4.6ds_data.txt"))

In [51]:
res_df_loaded = \
    pd.read_csv(shift.get_results_dir().fi('meas_FLAT_03_16_2020.csv'),
                index_col=0)
res_df_no_outliers = res_df_loaded[res_df_loaded['N']<2.4e7]
res_df_no_outliers = res_df_no_outliers.sort_values(by='N',ignore_index=True)

In [52]:
df_to_save = pd.DataFrame({"N": res_df_no_outliers['N']})

In [53]:
ey_func(df_to_save.iloc[0,0]),ey_func(df_to_save.iloc[-1,0])

(array(0.00474255), array(0.01143141))

In [55]:
len(res_df_no_outliers.index)

38

In [54]:
eys = np.linspace(0.003,0.013,10)

In [24]:
E_in, K_peak_in = (96.4, 1.0)
gamma_in = E_in/0.511
wr_sim = generate_wr_sim_with_wigrad_results(
K_peak_in=K_peak_in,
gamma_in=gamma_in)
ampx3d = wr_sim.get_amplittude_3D(polarization='x')
mesh = get_from_config("radiation_mesh")
zobs = get_from_config("z_obs_m")
xmin, xmax, _ = mesh[0]
xmin = xmin/zobs
xmax = xmax/zobs
ymin, ymax, _ = mesh[1]
ymin = ymin/zobs
ymax = ymax/zobs
lmin, lmax, _ = mesh[2]
dax = (xmax-xmin)/(mesh[0][2]-1)
day = (ymax-ymin)/(mesh[1][2]-1)
dl = (lmax-lmin)/(mesh[2][2]-1)
sm =dax*day*dl*np.sum(np.absolute(ampx3d)**2)

In [35]:
N0 = df_to_save.iloc[0,0]

In [37]:
ex = ex_func(N0)
ey = ey_func(N0)
dpp = dpp_func(N0)
st = sz_func(N0)
Sx, Sy, dx, dy, sxp, syp = CalcTransverseBeamParams(
    lattice_df,ex,ey,dpp)
coh_modes_cpp_args =  np.asarray([Sx,Sy,dx,dy,sxp,syp,xmin,xmax,ymin,ymax,lmin,lmax,
                                  st, sm, m0,mfold,seed],
                                dtype=np.float64)
Mi = np.real(
    cm.CalcMFromPrecalculatedFieldAmps(ampx3d,coh_modes_cpp_args)[-1])
print(Mi)

2307769.213356884
